In [2]:
import os; os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='1.' # NOTE: jax preallocates GPU (default 75%)
from pathlib import Path
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from jax import numpy as jnp, random as jr, config as jconfig, devices as jdevices, jit, vmap, pmap, grad, debug, tree, lax, value_and_grad
jconfig.update("jax_enable_x64", True)
print(jdevices())

%matplotlib inline
%load_ext autoreload
%autoreload 2
!hostname
#!jupyter nbconvert --to python /dvs_ro/u1/h/hsimfroy/workspace/montecosmo_proj/src/montecosmo/tests/infer_model2.ipynb

# save_dir = Path("./save/")
# save_dir = Path(os.path.expanduser("~/scratch/png/abacus_c0_i0_z08_lrg/test")) # FMN
# load_dir = Path(os.path.expanduser("~/scratch/png/abacus_c0_i0_z08_lrg/load/")) # FMN
# save_dir = Path("/lustre/fsn1/projects/rech/fvg/uvs19wt/png/") # JZ
# save_dir = Path("/lustre/fswork/projects/rech/fvg/uvs19wt/workspace/png/") # JZ
save_dir = Path("/pscratch/sd/h/hsimfroy/png/abacus_c0_i0_z0.8_lrg/noise") # Perlmutter
# load_dir = Path("/pscratch/sd/h/hsimfroy/png/abacus_c0_i0_z0.8_lrg/load/") # Perlmutter
load_dir = Path("/pscratch/sd/h/hsimfroy/png/fpm_b2760_z1_lrg_fNL/load/") # Perlmutter
# save_path = save_dir / "test"
# save_dir.mkdir(parents=True, exist_ok=True)

[CpuDevice(id=0)]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dphppce115.extra.cea.fr


In [ ]:
from montecosmo.model import FieldLevelModel, default_config
from montecosmo.utils import pdump, pload , chreshape, r2chshape, boxreshape, rg2cgh, cgh2rg
from montecosmo.bricks import top_hat_selection, gen_gauss_selection

box_size = 3*(2000,)
mesh_length = 96
z_obs = 0.8

model = FieldLevelModel(**default_config | 
                        {'final_shape': 3*(mesh_length,), 
                        'cell_length': box_size[0] / mesh_length, # in Mpc/h
                        'box_center': (0.,0.,0.), # in Mpc/h
                        # 'box_center': (0.,0.,1938.), # in Mpc/h # a2chi(model.cosmo_fid, a=1/(1+z_obs))

                        'box_rotvec': (0.,0.,0.,), # rotation vector in radians
                        # 'evolution': 'lpt',
                        'evolution': 'kaiser',
                        'a_obs': 1 / (1 + z_obs), # light-cone if None
                        'curved_sky': False, # curved vs. flat sky
                        'ap_auto': None, # parametrized AP vs. auto AP
                        'selection': None, # if float, padded fraction, if str or Path, path to window mesh file
                        'paint_order':2, # order of interpolation kernel
                        'paint_deconv': True, # whether to deconvolve painted field
                        'kernel_type':'rectangular', # 'rectangular', 'kaiser_bessel'
                        'init_oversamp':1.5, # initial mesh 1D oversampling factor
                        'evol_oversamp':2., # evolution mesh 1D oversampling factor
                        'ptcl_oversamp':2., # particle cloud 1D oversampling factor
                        'paint_oversamp':2., # painted mesh 1D oversampling factor

                        # 'init_oversamp':1.5, # initial mesh 1D oversampling factor
                        # 'evol_oversamp':1.5, # evolution mesh 1D oversampling factor
                        # 'ptcl_oversamp':1.5, # particle cloud 1D oversampling factor
                        # 'paint_oversamp':1.5, # painted mesh 1D oversampling factor

                        # 'init_oversamp':1., # initial mesh 1D oversampling factor
                        # 'evol_oversamp':1., # evolution mesh 1D oversampling factor
                        # 'ptcl_oversamp':1., # particle cloud 1D oversampling factor
                        # 'paint_oversamp':1., # painted mesh 1D oversampling factor
                        'interlace_order':2, # interlacing order
                        'n_rbins': 1,
                        'k_cut': np.inf,
                        'init_power': load_dir / f'init_kpow.npy',
                        # 'init_power': None,
                        # 'lik_type': 'gaussian_delta_power',
                        'lik_type': 'gaussian_fourier',
                        'png_type': 'fNL_bias',
                        } )

truth = {
    'Omega_m': 0.3137721, 
    'sigma8': 0.8076353990239834,
    'b1': 0.,
    'b2': 0.,
    'bs2': 0.,
    
    'b1': 0.,
    'b2': 0.,
    'bs2': 0.,
    'bn2': 0.,
    'bnpar': 0.,
    'fNL': 0.,
    'fNL_bp':0.,
    'fNL_bpd':0.,
    'alpha_iso': 1.,
    'alpha_ap': 1.,
    # 'ngbars': 8.43318125e-4,
    'ngbars': 1e-5,
    # 'ngbars': 10000., # neglect lik noise
    'sigma_0': 0.5,
    'sigma_2': 1e-6,
    'sigma_mu2': 1e-6,
    'sigma_delta': 0.7,
    }


latents = model.new_latents_from_loc(truth)
model = FieldLevelModel(**model.asdict() | {'latents': latents})
print(model)
# model.render()



# Abacus matter
# obs_mesh = jnp.load(load_dir / f'fin_paint2_interl2_deconv0_{mesh_length}.npy')
# obs_mesh = jnp.load(load_dir / f'fin_paint2_interl1_deconv1_{mesh_length}.npy')
obs_mesh = jnp.load(load_dir / f'fin_paint2_interl2_deconv1_{mesh_length}.npy')
obs_mesh *= truth['ngbars'] * model.cell_length**3

# Abacus initial
init_mesh = jnp.fft.rfftn(jnp.load(load_dir / f'init_mesh_{576}.npy'))
init_mesh = chreshape(init_mesh, r2chshape(model.init_shape))
truth0 = truth | {'init_mesh': init_mesh} | {'obs': obs_mesh}
del obs_mesh
del init_mesh
truth |= {'init_mesh': truth0['init_mesh']}

delta_sims = []
truths = []
confs = []
truths.append(truth)
confs.append({})
truths.append(truth)
confs.append({'evolution': 'lpt', 'lpt_order': 1,})
truths.append(truth)
confs.append({'evolution': 'lpt', 'lpt_order': 2,})
truths.append(truth)
confs.append({'evolution': 'nbody', 'lpt_order': 2,})


for tr, conf in zip(truths, confs):
    model = FieldLevelModel(**model.asdict() | conf)
    truth = model.predict(samples=tr, hide_base=False, hide_samp=False, from_base=True)
    delta_sim = model.count2delta(truth['obs'])
    delta_sims.append(delta_sim)

